# MNS - Biological Plausible Deep Learning
## Run Guergiev et al (2017) Pipeline

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from sklearn.model_selection import train_test_split

# Import log-helper/learning plot functions
from utils.helpers import *
from utils.plotting import *

# Guergiev et al (2017) - Segregated Compartments DNN Learning
from models.CompDNN import CompDNN, eval_comp_dnn
# Import Bayesian Optimization Module
from utils.bayesian_opt import BO_NN

In [3]:
# Create all necessary directory if non-existent
global data_dir
data_dir = os.getcwd() +"/data"

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print("Created New Data Directory")
    
download_data()

No download of MNIST needed.
No download of Fashion-MNIST needed.
No download of CIFAR-10 needed.


In [4]:
# MNIST dataset
X_mnist, y_mnist = get_data(num_samples=70000, dataset="mnist")

X_train, X_test, y_train, y_test = train_test_split(X_mnist, y_mnist,
                                                    stratify=y_mnist,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
n_epochs = 10
param_fname = "logs/comp_dnn_base_params.json"

net = CompDNN(X_train, y_train, param_fname)

net.train(n_epochs, log_freq=5000, verbose=True, logging=True, dataset="mnist")

Train| epoch  1| batch 5000/47995| acc: 0.8552| loss: 3.6680| time: 54.25
Valid| epoch  1| batch 5000/47995| acc: 0.8586| loss: 3.5581| time: 13.49
-------------------------------------------------------------------------
Train| epoch  1| batch 10000/47995| acc: 0.8933| loss: 2.8082| time: 55.28
Valid| epoch  1| batch 10000/47995| acc: 0.8930| loss: 2.8097| time: 8.27
-------------------------------------------------------------------------
Train| epoch  1| batch 15000/47995| acc: 0.9055| loss: 2.4238| time: 50.56
Valid| epoch  1| batch 15000/47995| acc: 0.9065| loss: 2.4106| time: 10.03
-------------------------------------------------------------------------
Train| epoch  1| batch 20000/47995| acc: 0.8923| loss: 2.5119| time: 51.39
Valid| epoch  1| batch 20000/47995| acc: 0.8922| loss: 2.5185| time: 9.54
-------------------------------------------------------------------------
Train| epoch  1| batch 25000/47995| acc: 0.9249| loss: 1.9485| time: 49.15
Valid| epoch  1| batch 25000/4799

In [5]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy MNIST: {}".format(acc))

Test Accuracy MNIST: 0.9713


In [6]:
# Fashion-MNIST dataset
X_fashion, y_fashion = get_data(num_samples=70000, dataset="fashion")

X_train, X_test, y_train, y_test = train_test_split(X_fashion, y_fashion,
                                                    stratify=y_fashion,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
n_epochs = 10
param_fname = "logs/comp_dnn_base_params.json"

net = CompDNN(X_train, y_train, param_fname)
net.train(n_epochs, log_freq=5000, verbose=True, logging=True, dataset="fashion")

Train| epoch  1| batch 5000/48000| acc: 0.6769| loss: 8.8360| time: 50.86
Valid| epoch  1| batch 5000/48000| acc: 0.6763| loss: 8.8637| time: 9.21
-------------------------------------------------------------------------
Train| epoch  1| batch 10000/48000| acc: 0.7575| loss: 6.1933| time: 51.14
Valid| epoch  1| batch 10000/48000| acc: 0.7540| loss: 6.2911| time: 9.53
-------------------------------------------------------------------------
Train| epoch  1| batch 15000/48000| acc: 0.7249| loss: 6.9574| time: 53.76
Valid| epoch  1| batch 15000/48000| acc: 0.7186| loss: 7.1510| time: 10.06
-------------------------------------------------------------------------
Train| epoch  1| batch 20000/48000| acc: 0.7696| loss: 6.0697| time: 50.76
Valid| epoch  1| batch 20000/48000| acc: 0.7672| loss: 6.2031| time: 9.83
-------------------------------------------------------------------------
Train| epoch  1| batch 25000/48000| acc: 0.7980| loss: 5.5254| time: 50.23
Valid| epoch  1| batch 25000/48000

In [7]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy Fashion-MNIST: {}".format(acc))

Test Accuracy Fashion-MNIST: 0.8476


In [8]:
# CIFAR-10 dataset
X_cifar10, y_cifar10 = get_data(num_samples=70000, dataset="cifar10")

X_train, X_test, y_train, y_test = train_test_split(X_cifar10, y_cifar10,
                                                    stratify=y_cifar10,
                                                    random_state=0,
                                                    test_size=1./7)

# set training parameters
n_epochs = 10
param_fname = "logs/comp_dnn_base_params.json"

net = CompDNN(X_train, y_train, param_fname)
net.train(n_epochs, log_freq=5000, verbose=True, logging=True, dataset="cifar10")

Train| epoch  1| batch 5000/41138| acc: 0.2064| loss: 21.9272| time: 190.39
Valid| epoch  1| batch 5000/41138| acc: 0.2056| loss: 21.9491| time: 30.62
-------------------------------------------------------------------------
Train| epoch  1| batch 10000/41138| acc: 0.2437| loss: 20.8976| time: 190.24
Valid| epoch  1| batch 10000/41138| acc: 0.2457| loss: 20.8428| time: 30.52
-------------------------------------------------------------------------
Train| epoch  1| batch 15000/41138| acc: 0.2275| loss: 21.3437| time: 192.07
Valid| epoch  1| batch 15000/41138| acc: 0.2311| loss: 21.2455| time: 30.36
-------------------------------------------------------------------------
Train| epoch  1| batch 20000/41138| acc: 0.2813| loss: 19.8504| time: 192.59
Valid| epoch  1| batch 20000/41138| acc: 0.2858| loss: 19.7289| time: 30.25
-------------------------------------------------------------------------
Train| epoch  1| batch 25000/41138| acc: 0.2472| loss: 20.7946| time: 192.40
Valid| epoch  1| 

In [9]:
X_test, y_test = prep_data_guergiev(X_test, y_test)
acc, loss = net.get_test_error(X_test, y_test)
print("Test Accuracy CIFAR-10: {}".format(acc))

Test Accuracy CIFAR-10: 0.34029398040130654


## Bayesian Optimization

In [20]:
eval_comp_dnn("mnist", param_fname, num_epochs=2, k_fold=3, verbose=True)

Dataset: mnist
Learning Rate Weights: 0.21
Architecture of Cross-Validated Network:
	 Layer 0: 500 Units
Cross-Validation Score Fold 1: 0.93585018854988
Cross-Validation Score Fold 2: 0.9394848497835684
Cross-Validation Score Fold 3: 0.8328404268998328


0.9027251550777603

In [33]:
# Run Bayesian Optimization (UCB-Acquisition Fct) on Compartmental DNN
hyper_space_comp_dnn = {"use_sparse_feedback": (0, 1),
                        "use_conductances": (0, 1),
                        "use_broadcast": (0, 1),
                        "use_spiking_feedback": (0, 1),
                        "use_spiking_feedforward": (0, 1),
                        "use_symmetric_weights": (0, 1),
                        "noisy_symmetric_weights": (0, 1),
                        "update_feedback_weights": (0, 1),
                        "use_apical_conductance": (0, 1),
                        "use_weight_optimization": (0, 1),
                        "use_feedback_bias": (0, 1),
                        
                        "dt" : (0.1, 1.0),
                        "l_f_phase": (2, 50),
                        "l_t_phase": (2, 50),
                        "l_f_phase_test": (50, 100),
                        
                        "lambda_max": (0.2, 0.5),
                        "tau_s": (1, 5),
                        "tau_L": (7, 13),
                        "g_B": (0.3, 0.9),
                        "g_A": (0.02, 0.1),
                        "E_E": (5, 12),
                        "E_I": (-12, -5),
                        
                        "f_etas": (0.01, 0.5),
                        "b_etas": (0.01, 0.5),
                        'num_layers': (1, 6),
                        
                        'h_l_1': (30, 500),
                        'h_l_2': (30, 500),
                        'h_l_3': (30, 500),
                        'h_l_4': (30, 500),
                        'h_l_5': (30, 500),
                        'h_l_6': (30, 500)}

# Test setup - run from run_bo.py file
bo_iters = 5
num_epochs = 2
k_fold = 2

In [ ]:
# Run Bayesian Optimization (UCB-Acquisition Fct) on CNN for MNIST
opt_log = BO_NN(bo_iters, eval_comp_dnn, "comp_dnn", "fashion", hyper_space_comp_dnn,
                num_epochs, k_fold, logging=True, verbose=True)

In [ ]:
# Run Bayesian Optimization (UCB-Acquisition Fct) on CompDNN for Fashion-MNIST
opt_log = BO_NN(bo_iters, eval_comp_dnn, "comp_dnn", "fashion", hyper_space_comp_dnn,
                num_epochs, k_fold, logging=True, verbose=True)

In [ ]:
# Run Bayesian Optimization (UCB-Acquisition Fct) on CompDNN for Cifar10
opt_log = BO_NN(bo_iters, eval_comp_dnn, "comp_dnn", "cifar10", hyper_space_comp_dnn,
                num_epochs, k_fold, logging=True, verbose=True)